# Time Series Forecasting for Capilano University Enrollment Data

This notebook processes enrollment data from Capilano University, converts it into time series format,
and applies various forecasting models (Seasonal Naive, ETS, ARIMA). The results are logged to
Weights & Biases for visualization and comparison.

Original Work: Jiaqi Li (jiaqili@capilanou.ca)  
Author: Eric Drechsler (dr.eric.drechsler@gmail.com)  
Version: 250301

## Environment Setup and Module Imports

This cell is responsible for setting up the Python environment and importing all
the necessary modules and libraries required for the time series analysis.

In [ ]:
# This cell sets up the environment, imports necessary modules, and configures
# logging

import logging
import os

import pyrootutils
from IPython.display import HTML, display

ROOT_PATH = pyrootutils.setup_root(
    search_from=os.getcwd(),
    indicator=".project_root",
    project_root_env_var=True,
    dotenv=True,
    pythonpath=True,
    cwd=True,
)

# This is the main function for processing time series data
from capu_time_series_analysis.utils import process_timeseries

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logger = logging.getLogger()

## Input Data

We rely on a datafile that contains enrollment data for Capilano University. The
data is stored in a CSV file which is part of the repository. The data is loaded
into a pandas DataFrame and displayed in the notebook.

In [ ]:
# After cloning the repository in your setup cell

# Define path to the input data file in the cloned repository
data_path = "data/input/capu_enrolment_data_200820-202310_v20230213.csv"

# Verify the file exists
import os

if os.path.exists(data_path):
    print(f"Found input data file at: {data_path}")

    # Preview the data
    import pandas as pd

    df = pd.read_csv(data_path)
    print(f"Data shape: {df.shape}")
    display(df.head())
else:
    print(f"Warning: Could not find data file at {data_path}")

## Configuration

This configuration dictionary controls the behavior of the time series analysis pipeline:

### Data Sources and Output Settings
- `input_file`: Specifies the path to the data file (set dynamically to `data_path`)
- `plot_dir`: Directory where generated plots will be saved (currently "output/plotssss")
- `save_plots`: Boolean flag (False) that controls whether plots are saved to disk

### Analysis Dimensions
- `levels`: Limits analysis to specific academic units ("CapU", "AS")
  - Commented code shows additional units that could be included (BPS, EHHD, FAA, GCS)
- `residencies`: Student types to analyze ("Domestic", "International")
- `metrics`: Data metrics to analyze (currently only "Headcount")
  - Commented code shows additional metrics that could be included (CourseEnrolment, AttemptedCredits)
- `forecast_steps`: Number of future time periods to forecast (9)

### Model Configuration
Parameters for each of the three forecasting models:

#### Seasonal Naive Model
- `seasonal_periods`: 3 (indicates quarterly/trimester data)

#### ETS (Exponential Smoothing State Space) Model
- `seasonal`: "add" (additive seasonality component)
- `damped_trend`: False (trend component is not dampened)

#### ARIMA (AutoRegressive Integrated Moving Average) Model
- `seasonal`: True (includes seasonal component)
- `m`: 3 (seasonal period length - quarters/trimesters)
- `d`: 0 (no differencing for trend component)
- `D`: 0 (no differencing for seasonal component)

The configuration focuses on analyzing enrollment headcount data across different university departments and student residency types, with models specifically tuned for trimester academic data patterns.

In [ ]:
# Configuration
cfg = {
    "input_file": data_path,
    "plot_dir": "output/plotssss",  # TODO exclude plot storage in ipynb
    "save_plots": False,
    # "levels": ["CapU", "AS"],
    "levels": ["CapU", "AS", "BPS", "EHHD", "FAA", "GCS"],
    "residencies": ["Domestic", "International"],
    # "metrics": ["Headcount"],
    "metrics": ["Headcount", "CourseEnrolment", "AttemptedCredits"],
    "forecast_steps": 9,
    "models": {
        "seasonal_naive_params": {"seasonal_periods": 3},
        "ets_params": {
            "seasonal": "add",
            "damped_trend": False,
        },
        "arima_params": {
            "seasonal": True,
            "m": 3,
            "d": 0,
            "D": 0,
        },
    },
}

## Data Processing

The time series processing is done inside the `process_timeseries` function, in
which data for different combinations of metrics, residencies, and levels is
analysed, forecasted, and evaluated.

This function processes each combination of metrics, residencies, and levels
through the following steps:

1. Loads data from the input file for the specific combination
2. Prepares and splits the time series into training and test sets
3. Fits three forecasting models: Seasonal Naive, ETS, and ARIMA
4. Analyzes residuals for each model to evaluate their fit quality
5. Generates forecasts for the test period to validate model performance
6. Creates future forecasts for the specified number of steps ahead
7. Evaluates forecast accuracy on the test set using multiple metrics
8. Consolidates all results into a single dataframe for easy analysis

The process is repeated for all combinations of metrics, residencies, and levels,
resulting in a comprehensive analysis across all specified dimensions.

In [ ]:
# Process time series
consolidated_df, evaluation_results, residual_diagnostics = process_timeseries(
    cfg["input_file"],
    cfg["metrics"],
    cfg["residencies"],
    cfg["levels"],
    forecast_steps=cfg["forecast_steps"],
    model_params=cfg["models"],
    save_plots=cfg["save_plots"],
    plots_dir=cfg["plot_dir"],
)

# Create evaluation dataframe
evaluation_df = pd.DataFrame(evaluation_results)
logger.info(f"Created evaluation dataframe with {len(evaluation_df)} entries")

# Create residual diagnostics dataframe
residual_df = pd.DataFrame(residual_diagnostics)
logger.info(f"Created residual diagnostics dataframe with {len(residual_df)} entries")

In [ ]:
train_filter = consolidated_df["Entry_Type"] == "Train"
test_filter = consolidated_df["Entry_Type"] == "Test"
actual_filter = consolidated_df["Model"] == "Actual"

In [ ]:
consolidated_df[test_filter & actual_filter].shape

In [ ]:
consolidated_df[test_filter]

### The Results DataFrame
All results are stored in a consolidated pandas DataFrame with the following columns: 

- `Analysis_Type`: Combination of metric, residency, and level
- `Residency`: Student type (Domestic, International)
- `Level`: Academic unit (CapU, AS, etc.)
- `Timestamp`: Date of the enrollment data
- `Model`: Forecasting model used (Seasonal Naive, ETS, ARIMA, Actual for input data)
- `Entry_Type`: Type of data (Train, Test, Forecast)
- `Entry`: Value of the data point


In [ ]:
# consolidated_df.value_counts(["Analysis_Type", "Residency", "Level", "Timestamp", "Model", "Entry_Type"],sort=False)
for col in ["Analysis_Type", "Residency", "Level", "Model", "Entry_Type"]:
    print(consolidated_df[col].value_counts())
    print()

## Analysing Results

The next section is concerned with analyzing the results of the time series
analysis. The functions below are specific to the notebook and are used to
generate plots and tables that summarize the results of the analysis.

Firstly, we import the necessary modules and libraries for the visualisation.
The plotting functions are defined in a helper module.

In [ ]:
# Visualization of Results
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from capu_time_series_analysis.plotting import (
    display_residual_plots,
    plot_evaluation_metrics,
    plot_time_series_combination,
)

# Set plotting style
plt.style.use("ggplot")
sns.set_theme(style="whitegrid")

## Time Series Visualization

This plot shows time series data for a specific combination of academic level, residency status, and metric. The visualization includes:

- **Blue Line**: Actual historical values used for training
- **Green Line**: Actual values in the test period 
- **Dashed Lines**: Model forecasts for the test period (allows comparison with actual values)
- **Dotted Lines**: Future forecasts beyond available data

This visualization helps you:
- Compare performance of different models (Seasonal Naive, ETS, ARIMA)
- Evaluate how well each model captures seasonal patterns and trends
- Identify which model produces the most realistic forecasts

Use the interactive dropdown menus to explore different combinations of data.

In [ ]:
# Create dropdown widget for interactive visualization if in Colab
try:
    from ipywidgets import Dropdown, interact

    # Get unique values for dropdowns
    levels_list = consolidated_df["Level"].unique().tolist()
    residencies_list = consolidated_df["Residency"].unique().tolist()
    metrics_list = consolidated_df["Analysis_Type"].unique().tolist()

    # Create interactive widget
    @interact(
        level=Dropdown(options=levels_list, description="Level:"),
        residency=Dropdown(options=residencies_list, description="Residency:"),
        metric=Dropdown(options=metrics_list, description="Metric:"),
    )
    def plot_interactive(level, residency, metric):
        plot_time_series_combination(consolidated_df, level, residency, metric)

except ImportError:
    # If not in an interactive environment, just plot a sample
    sample_level = consolidated_df["Level"].unique()[0]
    sample_residency = consolidated_df["Residency"].unique()[0]
    sample_metric = consolidated_df["Analysis_Type"].unique()[0]

    print(f"Plotting sample combination: {sample_level} - {sample_residency} - {sample_metric}")
    plot_time_series_combination(consolidated_df, sample_level, sample_residency, sample_metric)

## Model Evaluation Metrics

These bar charts compare the performance of each forecasting model (Seasonal Naive, ETS, ARIMA) across all combinations using standard error metrics:

- **RMSE (Root Mean Square Error)**: Emphasizes larger errors (lower is better)
- **MAE (Mean Absolute Error)**: Average absolute difference between predictions and actuals (lower is better)
- **MAPE (Mean Absolute Percentage Error)**: Average percentage error (lower is better)

The charts help identify:
- Which model performs best for each data combination
- Consistent patterns in model performance across different segments
- The magnitude of improvement between baseline and more complex models

In [ ]:
# Plot all three metrics
metrics = ["MAE", "RMSE", "MAPE"]
for metric in metrics:
    try:
        plot_evaluation_metrics(evaluation_df, metric)
        plt.show()
    except Exception as e:
        print(f"Error plotting {metric}: {e}")

In [ ]:
evaluation_df

## Residual Diagnostics

These plots examine the residuals (errors) of each model to assess their statistical validity:

- **Residuals Plot**: Shows residual values over time - ideally should look like random noise around zero
- **Actual vs. Fitted Plot**: Compares actual values with model predictions
- **Statistical Tests**: 
  - Mean Residual (should be close to zero)
  - Standard Deviation of Residuals
  - Ljung-Box p-value (tests for autocorrelation - p>0.05 suggests white noise)
  - White Noise Test result

Good models should have residuals that:
- Are randomly distributed with no patterns
- Have a mean close to zero
- Show no significant autocorrelation
- Pass the white noise test

These diagnostics help validate that a model has captured all significant patterns in the data.

In [ ]:
# Try to display residual diagnostics (if they include plots)
try:
    display_residual_plots(residual_df)
except Exception as e:
    print(f"Error displaying residual plots: {e}")

    # Fallback: Display residual statistics if plots aren't available
    print("\nFallback: Displaying residual statistics:")
    summary_stats = residual_df[
        [
            "Level",
            "Residency",
            "Analysis_Type",
            "Model",
            "Mean_Residual",
            "Std_Residual",
            "White_Noise",
        ]
    ].copy()
    display(summary_stats.head(10))

## Interactive Results Explorer

This is a stab at a comprehensive tool allows you to explore all aspects of the analysis in one place:

- **Time Series View**: Visualizes forecasts against actual values
- **Evaluation Metrics View**: Compares model performance metrics
- **Residual Diagnostics View**: Examines model validity through residual analysis

Use the dropdown menus to select:
- Academic level (Undergraduate, Graduate, etc.)
- Residency status (Domestic, International, etc.)
- Metric type (Headcount, FTE, etc.)
- View type (which aspect of the analysis to examine)

This interactive explorer makes it easy to drill down into specific combinations and understand model performance in detail.

In [ ]:
# 5. Interactive Results Explorer (works in Colab)
try:
    from ipywidgets import interact, widgets

    def explore_results(consolidated_df, evaluation_df, residual_df):
        """Interactive widget to explore all results"""

        levels = consolidated_df["Level"].unique().tolist()
        residencies = consolidated_df["Residency"].unique().tolist()
        metrics = consolidated_df["Analysis_Type"].unique().tolist()

        level_dropdown = widgets.Dropdown(options=levels, description="Level:")
        residency_dropdown = widgets.Dropdown(options=residencies, description="Residency:")
        metric_dropdown = widgets.Dropdown(options=metrics, description="Metric:")

        tab_dropdown = widgets.Dropdown(
            options=["Time Series", "Evaluation Metrics", "Residual Diagnostics"],
            description="View:",
        )

        def update_view(level, residency, metric, view):
            if view == "Time Series":
                plot_time_series_combination(consolidated_df, level, residency, metric)

            elif view == "Evaluation Metrics":
                # Filter evaluation df for this combination
                subset = evaluation_df[
                    (evaluation_df["Level"] == level)
                    & (evaluation_df["Residency"] == residency)
                    & (evaluation_df["Analysis_Type"] == metric)
                ]

                if not subset.empty:
                    # Create comparison bar chart
                    plt.figure(figsize=(10, 6))
                    metrics_to_plot = [
                        col for col in ["RMSE", "MAE", "MAPE"] if col in subset.columns
                    ]

                    for i, met in enumerate(metrics_to_plot):
                        plt.subplot(1, len(metrics_to_plot), i + 1)
                        sns.barplot(x="Model", y=met, data=subset)
                        plt.title(f"{met} by Model")
                        plt.xticks(rotation=45)

                    plt.tight_layout()
                    plt.show()

                    # Display table
                    display(subset)
                else:
                    print("No evaluation data for this combination")

            elif view == "Residual Diagnostics":
                # Filter residual df for this combination
                subset = residual_df[
                    (residual_df["Level"] == level)
                    & (residual_df["Residency"] == residency)
                    & (residual_df["Analysis_Type"] == metric)
                ]

                if not subset.empty:
                    for _, row in subset.iterrows():
                        model = row["Model"]
                        print(f"\n## {model} Model")

                        stats_info = (
                            f"Mean Residual: {row['Mean_Residual']:.4f}\n"
                            f"Std Residual: {row['Std_Residual']:.4f}\n"
                            f"Ljung-Box p-value: {row['Ljung_Box_pvalue']:.4f}\n"
                            f"White Noise Test: {'Passed' if row['White_Noise'] else 'Failed'}"
                        )
                        print(stats_info)

                        # Check if residual plots are available and display them
                        if "Residuals_Plot" in row and not pd.isna(row["Residuals_Plot"]):
                            display(
                                HTML(
                                    f'<img src="data:image/png;base64,{row["Residuals_Plot"]}" width="700px">'
                                )
                            )

                        if "Actual_Fitted_Plot" in row and not pd.isna(row["Actual_Fitted_Plot"]):
                            display(
                                HTML(
                                    f'<img src="data:image/png;base64,{row["Actual_Fitted_Plot"]}" width="700px">'
                                )
                            )
                else:
                    print("No residual data for this combination")

        # Create the interactive widget
        interact(
            update_view,
            level=level_dropdown,
            residency=residency_dropdown,
            metric=metric_dropdown,
            view=tab_dropdown,
        )

    # Use the interactive explorer
    explore_results(consolidated_df, evaluation_df, residual_df)

except ImportError:
    print("Interactive widgets are only available in Jupyter/Colab environments")
    # Provide a non-interactive alternative
    sample_level = consolidated_df["Level"].unique()[0]
    sample_residency = consolidated_df["Residency"].unique()[0]
    sample_metric = consolidated_df["Analysis_Type"].unique()[0]

    plot_time_series_combination(consolidated_df, sample_level, sample_residency, sample_metric)